In [5]:
import os
import numpy as np
np.random.seed(43)

In [212]:
setup_string = '''
# General setup
open morph-open-closed/open-closed.pdb
center
turn y 60

# Open bound structure get position of the ligand, missing in the morph
open 1ake
delete #1:.B
addh spec #1&protein hbond false
mmaker #0.1 #1

# Window appearance
windowsize 800 600
lighting reflectivity 0
set bg_color white
set silhouette
set silhouette_width 1.5

# Model appearance
color grey

# Add caption
2dlabel create title text "Binding and release of substrate leads to directional rotation of amino acid side chains" xpos 0.1 ypos 0.92 color black

# Play movie sequence
2dlabel change title visibility show

# Hide everything
~ribbon; ~display
# Except ligand
display #1:AP5; color red #1:AP5; color byhet #1:AP5
'''

In [213]:
def pause_and_fade(pause_length, total_frames, file):
    file.write('# Fade out...\n')
    for pause in np.arange(pause_length):
        total_frames += 1
        transparency_increment = 100. / float(pause_length)
        file.write('transparency {} #1:AP5\n'.format((pause + 1)* transparency_increment)
                  )
        file.write('copy file morph-open-closed/open-closed-{:03.0f}.png\n'.format(total_frames)
        )
    file.write('~display #1:AP5\n')
    return total_frames

In [214]:
def pause_and_fade_in(pause_length, total_frames, file):
    file.write('# Fade in...\n')
    file.write('display #1:AP5\n')
    for pause in np.arange(pause_length):
        total_frames += 1
        transparency_increment = 100. / float(pause_length)
        file.write('transparency {} #1:AP5\n'.format(100 - ((pause + 1)* transparency_increment))
        )
        file.write('copy file morph-open-closed/open-closed-{:03.0f}.png\n'.format(total_frames)
        )

    return total_frames


def pause(pause_length, total_frames, file):
    for pause in np.arange(pause_length):
        total_frames += 1
        file.write('copy file morph-open-closed/open-closed-{:03.0f}.png\n'.format(total_frames)
        )
    return total_frames

In [215]:
def add_bump(positive):
    sigma = 2
    if positive:
        mu = 6.0
    else:
        mu = -6.0
    rotation = sigma * np.random.randn(1) + mu
    return rotation[0]

In [216]:
def rotate_forward(models, residues, positives, 
                   list_of_previous_positions, file, colors, total_frames):
    #
    positions = np.empty(( len(residues), len(models) ))
    #
    for model in models:
        file.write('~show #0; ~ribbon #0\n')
        for (residue_index, residue) in enumerate(residues): 
            
            if model == models[0]:

                positions[residue_index, model] = \
                                list_of_previous_positions[residue_index, model] + \
                                add_bump(positives[residue_index])
            else:
                positions[residue_index, model] = \
                                positions[residue_index, model - 1] + \
                                add_bump(positives[residue_index])

            additional_rotation = positions[residue_index, model] - \
                                  list_of_previous_positions[residue_index, model]

            
            file.write('rotation 2 #0.{}:{}@CA,CB\n'.format(model + 1, residue))
            file.write('rotation 2 {} 1; wait 1\n'.format(additional_rotation))
            file.write('display #0.{}:{};'.format(model + 1, residue))
            file.write('color {} #0.{}:{};'.format(colors[residue_index], model + 1, residue))
            file.write('color byhet #0.{}:{}\n'.format(model + 1, residue))
            file.write('~rotation 2\n')


            print('Frame {:02d}: {:.2f} → {:.2f}, which is a rotation of additional {:.2f}'.format(model + 1, 
                                                               list_of_previous_positions[residue_index, model],
                                                               positions[residue_index, model],
                                                               additional_rotation))
            
        file.write('ribbon #0.{}\n'.format(model + 1))
        total_frames += 1
        file.write('copy file morph-open-closed/open-closed-{:03.0f}.png \n\n'.format(total_frames)
        )

    return positions, total_frames

In [217]:
def rotate_backward(models, residues, positives, list_of_previous_positions, file, colors, total_frames):
    #
    positions = np.zeros(( len(residues), len(models) + 1))
    #
    for model in models[::-1]:
        file.write('~show #0; ~ribbon #0\n')
        for (residue_index, residue) in enumerate(residues):
            
            if model == models[-1]:

                positions[residue_index, model] = \
                                list_of_previous_positions[residue_index, model] + \
                                add_bump(positives[residue_index])
            else:
                positions[residue_index, model] = \
                                positions[residue_index, model + 1] + \
                                add_bump(positives[residue_index])
#                 print(f'{positions[residue_index, model]:.2f} = {positions[residue_index, model + 1]:.2f} + ' + \
#                       f'{add_bump(positives[residue_index]):.2f}')

            additional_rotation = positions[residue_index, model] - \
                                  list_of_previous_positions[residue_index, model]
            
            print('Frame {:02d}: {:.2f} → {:.2f}, which is a rotation of additional {:.2f}'.format(model + 1, 
                                                                               list_of_previous_positions[residue_index, model],
                                                                               positions[residue_index, model],
                                                                               additional_rotation))   
            
            file.write('rotation 2 #0.{}:{}@CA,CB\n'.format(model + 1, residue))
            file.write('rotation 2 {} 1; wait 1\n'.format(additional_rotation))
            file.write('display #0.{}:{};'.format(model + 1, residue))
            file.write('color {} #0.{}:{};'.format(colors[residue_index], model + 1, residue))
            file.write('color byhet #0.{}:{}\n'.format(model + 1, residue))
            file.write('~rotation 2\n')


        file.write('ribbon #0.{}\n'.format(model + 1))
        total_frames += 1
        file.write('copy file morph-open-closed/open-closed-{:03.0f}.png \n\n'.format(total_frames)
        )
    
    return positions, total_frames

In [219]:
residues = [20, 50, 120, 141, 200]
positives = [True, False, False, True, True]
pause_length = 20
morphs = np.arange(30)
total_frames = 0
colors = ['blue', 'green', 'purple', 'orange', 'yellow']



with open('morph_setup.cmd', 'w') as file:
    file.write(setup_string)

    forward_rotations, total_frames = rotate_forward(
        models=morphs,
        residues=residues,
        positives=positives,
        list_of_previous_positions=np.zeros(( len(residues), len(morphs) + 1)),
        file=file, 
        colors=colors,
        total_frames=total_frames)
    
    total_frames = pause_and_fade(5, total_frames, file)
    print('')
    backward_rotations, total_frames = rotate_backward(
        models=morphs, 
        residues=residues, 
        positives=positives, 
        list_of_previous_positions=forward_rotations,
        colors=colors,
        file=file, 
        total_frames=total_frames)
    
    total_frames = pause_and_fade_in(5, total_frames, file)
    print('')
    forward_rotations, total_frames = rotate_forward(
        models=morphs,
        residues=residues,
        positives=positives, 
        list_of_previous_positions=backward_rotations,
        file=file, 
        colors=colors,
        total_frames=total_frames)

    total_frames = pause_and_fade(5, total_frames, file)
    print('')
    backward_rotations, total_frames = rotate_backward(
        models=morphs, 
        residues=residues, 
        positives=positives, 
        list_of_previous_positions=forward_rotations,
        colors=colors,
        file=file, 
        total_frames=total_frames)

    total_frames = pause(5, total_frames, file)

Frame 01: 0.00 → 4.19, which is a rotation of additional 4.19
Frame 01: 0.00 → -8.11, which is a rotation of additional -8.11
Frame 01: 0.00 → -6.21, which is a rotation of additional -6.21
Frame 01: 0.00 → 4.44, which is a rotation of additional 4.44
Frame 01: 0.00 → 5.35, which is a rotation of additional 5.35
Frame 02: 0.00 → 9.93, which is a rotation of additional 9.93
Frame 02: 0.00 → -10.87, which is a rotation of additional -10.87
Frame 02: 0.00 → -10.43, which is a rotation of additional -10.43
Frame 02: 0.00 → 11.18, which is a rotation of additional 11.18
Frame 02: 0.00 → 8.18, which is a rotation of additional 8.18
Frame 03: 0.00 → 19.59, which is a rotation of additional 19.59
Frame 03: 0.00 → -20.86, which is a rotation of additional -20.86
Frame 03: 0.00 → -17.43, which is a rotation of additional -17.43
Frame 03: 0.00 → 18.54, which is a rotation of additional 18.54
Frame 03: 0.00 → 18.90, which is a rotation of additional 18.90
Frame 04: 0.00 → 26.84, which is a rotatio

```
ffmpeg -r 10 -i open-closed-%03d.png -y -vf format=yuv420p morph.mp4
```

I probably need to create a function that goes forward, just starting with an initial value -- like I did for the backwards process, but without stepping through the frames backwards. In fact, I should just make this the general going forwards function and pass an array of zeros for the first trip forwards.

In [121]:
[[0] * len(residues), [0] * len(morphs)]

[[0], [0, 0, 0, 0, 0]]